In [2]:
%matplotlib inline
import importlib
import utils2; importlib.reload(utils2)
from utils2 import *
np.set_printoptions(4)
PATH = '/home/ubuntu/data/spellbee/'

In [3]:
limit_mem()

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
lines = [l.strip().split("  ") for l in open(PATH+"cmudict-0.7b", encoding='latin1') 
         if re.match('^[A-Z]', l)]

In [6]:
lines=[(w, ps.split()) for w, ps in lines]
lines[0], lines[-1] 

(('A', ['AH0']), ('ZYWICKI', ['Z', 'IH0', 'W', 'IH1', 'K', 'IY0']))

In [7]:
len(lines)

133779

In [8]:
phonemes=["_"] + sorted(set(p for w, ps in lines for p in ps))
phonemes[:5]

['_', 'AA0', 'AA1', 'AA2', 'AE0']

In [9]:
len(phonemes)

70

In [10]:
'''Then we create mappings of phonemes and letters to respective indices.
Our letters include the padding element "_", but also "*" which we'll explain later.'''
p2i = dict((v, k) for k, v in enumerate(phonemes))
letters="_abcdefghijklmnopqrstuvwxyz*"
l2i=dict((v,k) for k, v in enumerate(letters))

In [11]:
len(p2i), len(l2i)

(70, 28)

In [12]:
'''create a dictionary mapping words to the sequence of indices corresponding to it's phonemes, 
only for words between 5 - 15 characters'''
maxlen=15
pronounce_dict={w.lower(): [p2i[p] for p in ps] for w, ps in lines if (5<=len(w)<=maxlen) and re.match("^[A-Z]+$",w)}


In [13]:
# for w, ps in lines[:20]:
#     if (5<=len(w)<=maxlen) and re.match("^[A-Z]+$",w):
# #         print(w, ps)
#         print(w.lower())
#         for p in ps:
#             print(p, p2i[p])

In [14]:
len(pronounce_dict)

108006

In [15]:
a = ['xyz','abc']
[o.upper() for o in a if o[0]=='x']
[[p for p in o] for o in a]
[p for o in a for p in o]

['x', 'y', 'z', 'a', 'b', 'c']

In [16]:
# Split lines into words, phonemes, convert to indexes (with padding), split into training, validation, test sets. 
# Note we also find the max phoneme sequence length for padding.

In [17]:
maxlen_p = max([len(v) for k, v in pronounce_dict.items()])

In [18]:
maxlen_p

16

In [19]:
pairs=np.random.permutation(list(pronounce_dict.keys()))

In [20]:
n=len(pairs)
input_=np.zeros((n, maxlen_p), np.int32)
labels_=np.zeros((n, maxlen), np.int32)
input_.shape, labels_.shape, n

((108006, 16), (108006, 15), 108006)

In [21]:
pronounce_dict['thumbed']

[58, 8, 44, 21]

In [22]:
for i , k in enumerate(pairs):
    for j, p in enumerate(pronounce_dict[k]): 
#         print(j, p)
        input_[i][j]=p
    for j, letter in enumerate(k): 
#         print(j, letter)
        labels_[i][j]= l2i[letter]
        
input_.shape

(108006, 16)

In [23]:
go_token=l2i["*"]
dec_input_=np.concatenate([np.ones((n,1)) * go_token, labels_[:,:-1]], axis=1)

In [24]:
(np.ones((n,1)) * go_token).shape,labels_[:,:-1].shape

((108006, 1), (108006, 14))

In [25]:
(input_train, input_test, labels_train, labels_test, 
 dec_input_train, dec_input_test) = train_test_split(input_, 
                    labels_, dec_input_, test_size=0.1)

In [26]:
input_train.shape,labels_train.shape

((97205, 16), (97205, 15))

In [27]:
input_vocab_size, output_vocab_size = len(phonemes), len(letters)
input_vocab_size, output_vocab_size

(70, 28)

## start building model

In [28]:
parms={'verbose':0, 'callbacks':[TQDMNotebookCallback(leave_inner=True)]}
lstm_params={}
dim=240

### without attention

In [29]:
def get_rnn(return_sequences=True):
    return LSTM(dim, dropout=0.1,recurrent_dropout=0.1, 
                implementation=2,
                return_sequences=return_sequences)

In [30]:
maxlen

15

In [31]:
# get_rnn return sequence = True default 

inp = Input((maxlen_p,))

print(inp.shape)
x = Embedding(input_vocab_size, 120)(inp)

print(x.shape)
x = Bidirectional(get_rnn())(x)
print(x.shape)
x = get_rnn(False)(x)
print(x.shape)
x = RepeatVector(maxlen)(x)
print(x.shape)
x = get_rnn()(x)
print(x.shape)
x=get_rnn()(x)
print(x.shape)
x=TimeDistributed(Dense(output_vocab_size, activation='softmax'))(x)
print(x.shape)


(?, 16)
(?, 16, 120)
(?, ?, 480)
(?, 240)
(?, 15, 240)
(?, ?, 240)
(?, ?, 240)
(?, 15, 28)


In [32]:
model = Model(inp, x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 16, 120)           8400      
_________________________________________________________________
bidirectional_1 (Bidirection (None, 16, 480)           693120    
_________________________________________________________________
lstm_2 (LSTM)                (None, 240)               692160    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 15, 240)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 15, 240)           461760    
_________________________________________________________________
lstm_4 (LSTM)                (None, 15, 240)           461760    
__________

In [33]:
model.compile(Adam(), 'sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
hist=model.fit(input_train, np.expand_dims(labels_train,-1), 
          validation_data=[input_test, np.expand_dims(labels_test,-1)], 
          batch_size=64, **parms, nb_epoch=3)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


97152/|/[loss: 1.517, acc: 0.553] 100%|| 97152/97205 [15:00<00:00, 110.22it/s]

97152/|/[loss: 1.047, acc: 0.667] 100%|| 97152/97205 [14:59<00:00, 111.07it/s]

In [ ]:
hist.history['val_loss']

In [ ]:
def eval_keras(input):
    preds = model.predict(input, batch_size=128)
    predict = np.argmax(preds, axis = 2)
    return (np.mean([all(real==p) for real, p in zip(labels_test, predict)]), predict)

In [ ]:
acc, preds = eval_keras(input_test); acc

In [40]:
def print_examples(preds):
    print("pronunciation".ljust(40), "real spelling".ljust(17), 
          "model spelling".ljust(17), "is correct")

    for index in range(20):
        ps = "-".join([phonemes[p] for p in input_test[index]]) 
        real = [letters[l] for l in labels_test[index]] 
        predict = [letters[l] for l in preds[index]]
        print (ps.split("-_")[0].ljust(40), "".join(real).split("_")[0].ljust(17),
            "".join(predict).split("_")[0].ljust(17), str(real == predict))

In [42]:
print_examples(preds)

pronunciation                            real spelling     model spelling    is correct
K-AE1-K-AH0-L                            cackle            cackll            False
D-IH0-M-AA1-R-IY0-AH0                    demaria           dimaria           False
M-AH0-K-L-EH1-L-AH0-N-D                  mcclelland        mclleland         False
D-R-AH0-M-AE1-T-IH0-K-S                  dramatics         dramattis         False
Z-AY1-L-IH0-N-AH0                        xylina            zilina            False
S-EH1-N                                  senne             senne             True
S-AH0-B-S-IH1-S-T                        subsist           subsist           True
D-IH0-T-IH1-R-IY0-ER0-EY2-T-S            deteriorates      deterrrrattt      False
B-L-EH1-R                                blare             bleer             False
M-EH1-L-K-AA2-R                          melkar            milcar            False
T-EH2-K-T-R-AA1-N-IH0-K-S                tektronix         tectranicc        False
N

# with attention

In [1]:
import attention_wrapper; importlib.reload(attention_wrapper)
from attention_wrapper import Attention
# teacher forcing

Using TensorFlow backend.


NameError: name 'importlib' is not defined

In [32]:
inp = Input((maxlen_p,))
inp_dec = Input((maxlen,))
print(inp.shape, inp_dec.shape)
emb_dec = Embedding(output_vocab_size, 120)(inp_dec)
emb_dec = Dense(dim)(emb_dec)


(?, 16) (?, 15)


In [37]:
?Attention()

In [36]:
x= Embedding(input_vocab_size, 120)(inp)
print(x.shape)
x= Bidirectional(get_rnn())(x)
print(x.shape)
x = get_rnn()(x)
print(x.shape)
x = get_rnn()(x)
print(x.shape)
x = Attention(get_rnn, 3)([x, emb_dec])

(?, 16, 120)
(?, ?, 480)
(?, ?, 240)
(?, ?, 240)
